# Вариант 2

1. построить модель линейной регрессии с указанными параметрами своими средствами (вместе со свободным коэффициентом), то есть пользоваться готовыми линейными моделями нельзя (максимум – для сравнения своего результата с готовой реализации, плюс можно использовать NumPy для матричных вычислений или специальные функции, решающие оптимизационные задачи);
2. рассчитать оценки наименьших квадратов для параметров и остаточной дисперсии;
3. вычислить коэффициент детерминации;
4. построить доверительные интервалы для параметров модели и остаточной дисперсии;


Зависимая переменная – цена, предикторы – мощность, емкость бензобака, расход топлива в город


In [2]:
import numpy as np
import pandas as pd
from scipy.stats import t, chi2
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


data = pd.read_csv("cars93.csv")
dependent_var = np.array(data["Price"])
independent_vars = np.array([data["MPG.city"], data["Horsepower"], data["Fuel.tank.capacity"], np.ones(len(dependent_var))]).T

# Находим коэффициенты методом наименьших квадратов для линейной регрессии
coeffs = np.linalg.inv(independent_vars.T @ independent_vars) @ independent_vars.T @ dependent_var
print("Вектор параметров вычисленный самостоятельно:", coeffs)


# Готовая реализация
model = LinearRegression().fit(independent_vars, dependent_var)
print("Вектор параметров вычисленный при помощи библиотеки:", np.concatenate((model.coef_[:-1],  np.array([model.intercept_]))))
# print(model.score(independent_vars, dependent_var))


m = 3
n = len(data)
ss_tot = np.sum((dependent_var - np.mean(dependent_var))**2) # Общая сумма квадратов
ss_reg = np.sum((np.dot(coeffs, independent_vars.T) - np.mean(dependent_var))**2) # Объясненная сумма квадратов
ss_res = ss_tot - ss_reg # Сумма квадратов остатков регрессии
varince = ss_res / (n - m)
print("Коэффициент детерминации:", ss_reg / ss_tot)
print("Оценка дисперсии:", varince)

# Вычисление ковариантной матирцы
cov_matrix = np.linalg.inv(independent_vars.T @ independent_vars)
standart_errors = np.sqrt(varince * np.diag(cov_matrix))

# Вычисление критических точек
alpha = 0.05
t_value = abs(t.ppf(alpha / 2, n - m))
print("Доверительные итнервалы для параметров модели:")

for i in range(len(coeffs)):
    print([coeffs[i] - standart_errors[i] * t_value, coeffs[i] + standart_errors[i] * t_value])

print("Доверительный интервал для остаточной дисперсии:")
print(varince * (n - m) / chi2.ppf(1 - alpha / 2, n - m), varince * (n - m) / chi2.ppf(alpha / 2, n - m))


Вектор параметров вычисленный самостоятельно: [-0.21153346  0.13197168 -0.12880216  5.60314185]
Вектор параметров вычисленный при помощи библиотеки: [-0.21153346  0.13197168 -0.12880216  5.60314185]
Коэффициент детерминации: 0.628928390835737
Оценка дисперсии: 35.39207325889215
Доверительные итнервалы для параметров модели:
[-0.5400157402149558, 0.11694882978844046]
[0.09553403363791829, 0.16840932913785156]
[-2.0619262124869526, 1.8043218927749733]
[-6.244693784029669, 17.450977492835815]
Доверительный интервал для остаточной дисперсии:
26.96290284230023 48.521716897140806
